You are given an integer n. There are n rooms numbered from 0 to n - 1.

You are given a 2D integer array meetings where meetings[i] = [starti, endi] means that a meeting will be held during the half-closed time interval [starti, endi). All the values of starti are unique.

Meetings are allocated to rooms in the following manner:

Each meeting will take place in the unused room with the lowest number.
If there are no available rooms, the meeting will be delayed until a room becomes free. The delayed meeting should have the same duration as the original meeting.
When a room becomes unused, meetings that have an earlier original start time should be given the room.
Return the number of the room that held the most meetings. If there are multiple rooms, return the room with the lowest number.

A half-closed interval [a, b) is the interval between a and b including a and not including b.

 

Example 1:

Input: n = 2, meetings = [[0,10],[1,5],[2,7],[3,4]]
Output: 0
Explanation:
- At time 0, both rooms are not being used. The first meeting starts in room 0.
- At time 1, only room 1 is not being used. The second meeting starts in room 1.
- At time 2, both rooms are being used. The third meeting is delayed.
- At time 3, both rooms are being used. The fourth meeting is delayed.
- At time 5, the meeting in room 1 finishes. The third meeting starts in room 1 for the time period [5,10).
- At time 10, the meetings in both rooms finish. The fourth meeting starts in room 0 for the time period [10,11).
Both rooms 0 and 1 held 2 meetings, so we return 0. 
Example 2:

Input: n = 3, meetings = [[1,20],[2,10],[3,5],[4,9],[6,8]]
Output: 1
Explanation:
- At time 1, all three rooms are not being used. The first meeting starts in room 0.
- At time 2, rooms 1 and 2 are not being used. The second meeting starts in room 1.
- At time 3, only room 2 is not being used. The third meeting starts in room 2.
- At time 4, all three rooms are being used. The fourth meeting is delayed.
- At time 5, the meeting in room 2 finishes. The fourth meeting starts in room 2 for the time period [5,10).
- At time 6, all three rooms are being used. The fifth meeting is delayed.
- At time 10, the meetings in rooms 1 and 2 finish. The fifth meeting starts in room 1 for the time period [10,12).
Room 0 held 1 meeting while rooms 1 and 2 each held 2 meetings, so we return 1. 
 

Constraints:

1 <= n <= 100
1 <= meetings.length <= 105
meetings[i].length == 2
0 <= starti < endi <= 5 * 105
All the values of starti are unique.

# approach 1:
- thinking of using 3 heaps, - one for meeting room tracking, one for running meetings, one for delayed meetings.
- go through the meetings and assign the meeting to the available room push it to the running meeting (end time, room no) - min heap
- then if any meeting gets over, push back the room to the available room.
- if rooms are avilable then strat using it.

In [ ]:
import heapq

class Solution:
    def mostBooked(self, n: int, meetings: list[list[int]]) -> int:
        meetings.sort()
        available_rooms = list(range(n))
        heapq.heapify(available_rooms)
        running_meetings = [] # (end time, room no) - min heap
        delayed_meetings = [] # (start, end time) - min heap 
        how_many_meetings = {} # room no: freq of used for meeting
        max_end_time = 0 

        # push the meetings to the respective heaps.
        for m in meetings:
            if available_rooms:
                # mark the room when there is a meeting happening.
                room_no = heapq.heappop(available_rooms)
                how_many_meetings[room_no] = how_many_meetings.get(room_no, 0) + 1
                heapq.heappush(running_meetings, (m[1], room_no))
            else:
                heapq.heappush(delayed_meetings, m)

            if m[1] > max_end_time:
                max_end_time = m[1]

        print(available_rooms, running_meetings, how_many_meetings)
        
        # start ending the meetings.
        for end in range(max_end_time + 1):
            while running_meetings and running_meetings[0][0] <= end:
                # this meeting is done, we can give back the room to available.
                t, room_no = heapq.heappop(running_meetings)
                heapq.heappush(available_rooms, room_no)
            
            # when there is some available rooms, we can use them.
            while available_rooms and delayed_meetings:
                s,e = heapq.heappop(delayed_meetings)
                # The meeting duration should ne the original meeting duration.
                duration = e - s 
                # so the new meeting will start from the current end time and end at end + druration time.
                room_no = heapq.heappop(available_rooms)
                how_many_meetings[room_no] = how_many_meetings.get(room_no, 0) + 1
                heapq.heappush(running_meetings, (end + duration, room_no))

        # Find room with maximum meeting count, return smallest index in case of tie
        print(how_many_meetings)
        max_meetings = max(how_many_meetings.values())
        for i in range(n):
            if how_many_meetings[i] == max_meetings:
                return i


# Use the below code. for better one. NOTE

In [22]:
Solution().mostBooked(n = 2, meetings = [[0,10],[1,5],[2,7],[3,4]])

[] [(5, 1), (10, 0)] {0: 1, 1: 1}
{0: 1, 1: 2}


1

In [23]:
Solution().mostBooked(n = 3, meetings = [[1,20],[2,10],[3,5],[4,9],[6,8]])

[] [(5, 2), (20, 0), (10, 1)] {0: 1, 1: 1, 2: 1}
{0: 1, 1: 2, 2: 2}


1

In [24]:
Solution().mostBooked(n=1, meetings = [[0,1]])

[] [(1, 0)] {0: 1}
{0: 1}


0

In [ ]:
import heapq

class Solution:
    def mostBooked(self, n: int, meetings: list[list[int]]) -> int:
        # meetings.sort()
        available = list(range(n))
        heapq.heapify(available)
        running = []  # (end_time, room_number)
        count = [0] * n  # count[i] = number of meetings room i handled

        for start, end in meetings:
            # Free up rooms whose meetings ended before current meeting's start
            while running and running[0][0] <= start:
                _, room = heapq.heappop(running)
                heapq.heappush(available, room)
            
            if available:
                # if there are rooms available, take it.
                room = heapq.heappop(available)
                heapq.heappush(running, (end, room))
                count[room] += 1
            else:
                # No rooms free, Take the earlist one and push the current meeting hte new end time duration.
                earliest_end, room = heapq.heappop(running)
                duration = end - start
                new_end = earliest_end + duration
                heapq.heappush(running, (new_end, room))
                count[room] += 1
        
        # Find room with maximum meeting count, return smallest index in case of tie
        max_meetings = max(count)
        for i in range(n):
            if count[i] == max_meetings:
                return i


# tc :
# - sorting O(m log m)
# - For each meeting: heap ops → O(log n)

# sc:
# O(n) for available rooms and count array
# O(m) for heap (in worst case)


n = 3
meetings = [[1, 20], [2, 10], [3, 5], [4, 9], [6, 8]]


- how to calculate the new end time. = (5, 2) 5 end time running in room no 2. . (4,9) is comming, 
- so we will end the meeting at 5. so the 4th meeting can start at 5 and the duration of the meeting is (9 - 4) == 5. 
- so it wiil start at 5 and end at 10.



| Step | Meeting  | Action                                                             | Room Used | New End Time | Running Heap (after)          | Available Heap (after) | Count       |
| ---- | -------- | ------------------------------------------------------------------ | --------- | ------------ | ----------------------------- | ---------------------- | ----------- |
| 1    | \[1, 20] | Room 0 available, schedule directly                                | 0         | 20           | `[(20, 0)]`                   | `[1, 2]`               | `[1, 0, 0]` |
| 2    | \[2, 10] | Room 1 available, schedule directly                                | 1         | 10           | `[(10, 1), (20, 0)]`          | `[2]`                  | `[1, 1, 0]` |
| 3    | \[3, 5]  | Room 2 available, schedule directly                                | 2         | 5            | `[(5, 2), (20, 0), (10, 1)]`  | `[]`                   | `[1, 1, 1]` |
| 4    | \[4, 9]  | No rooms free, wait for (5, 2), delay to \[5, 10], re-use room 2   | 2         | 10           | `[(10, 1), (20, 0), (10, 2)]` | `[]`                   | `[1, 1, 2]` |
| 5    | \[6, 8]  | No rooms free, wait for (10, 1), delay to \[10, 12], re-use room 1 | 1         | 12           | `[(10, 2), (20, 0), (12, 1)]` | `[]`                   | `[1, 2, 2]` |

